# Face ID Detection With Custom Data

**References Source: https://github.com/mdsarfarazulh/face-verification-facenet-mtcnn**

### Install necessary package

In [1]:
!pip install mtcnn
# Note! Skiping step when having this available package

In [2]:
!pip install tensorflow

### Importing necessaty librarys

In [1]:
# example of using the vgg16 model as a feature extraction model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from pickle import dump
import os
import cv2
import numpy as np
import cv2
import numpy as np 

from mtcnn.mtcnn import MTCNN
import pickle
import os.path

### Preprocessing image and Extracting feature

In [2]:
def preproces_image(path):
    
  # load an image from file
    image = load_img(path, target_size=(224, 224))
    image = image.resize((224,224))
  # convert the image pixels to a numpy array
    image = img_to_array(image)
  # reshape data for the model
  #image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = np.expand_dims(image, axis = 0)
  # prepare the image for the VGG model
    image = preprocess_input(image)
    return image

def feature_extract(model, image):
    features = model.predict(image)[0]
  # Vector normalization
    features = features / np.linalg.norm(features)
    return features

### This case, we you VGG16 model pre-train to extracting features.

# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [3]:
# Using MTCNN model as detector to detecting face in each frame
detector = MTCNN()

# Upload file contain vector of face previous extracted
p_file = "vectors.pkl"

# Set some parameters
font = cv2.FONT_HERSHEY_DUPLEX
color = (0, 255, 255)
line = cv2.LINE_AA

# Setting the threshold value.
threshold = 15.0

def get_euclidean(X, Y):
    return np.sqrt(np.sum(np.square(np.subtract(X, Y))))

with open(p_file, 'rb') as f:
    data_dict = pickle.load(f)
print('Loading data!')

# Label of each face in list vector, sort by order
Y = ["Quan", "Phuong", "Truong", "Thuan", "Phuc"]

Loading data!


In [ ]:
# Opening a video source either file or webcam
cap = cv2.VideoCapture(0)
# Checking whether video source opened or not.
if not cap.isOpened():
    print('Video not opened!')


# Operating until video source is present.
while cap.isOpened():

    _, frame  = cap.read()
    ##############################

    faces = detector.detect_faces(frame)
    
    for face in faces:
        # Getting the co-ordinates of the bounding box.
        x, y, w, h = face['box']
        # Getting Region Of Image(ROI)
        f_img = frame[y:y+h, x:x+w]
        # Resizing the face in the shape of (self.width, self.height)
        f_img = cv2.resize(f_img, (224, 224))
        # Calling the helper function to get the label.
        
        ##################################
        f_img = f_img.astype(np.float16).reshape((1, 224, 224, 3))
        # Normalizing the data to reduce computing time and memory.
        f_img /= 255.0
        
        feature = feature_extract(model, f_img)
        dist = []
        
        # Calculating euclidean distance.
        for vector in data_dict:
            dist.append(get_euclidean(vector, feature))
        
        print(dist)
        dist = np.array(dist)
        # Getting the most similar face.
        indx = np.argmin(dist)
        
        if dist[indx] < threshold:
            label = Y[indx]
        else:
            label =  "Opps!" 
        ###################################
        # Drawing rectangle and putting text on the bounding box of each fce
        cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2, line)
        cv2.putText(frame, label, (x-3, y-3), font, 0.5, color, 1)
    
    ##############################
    cv2.imshow('Frame', frame)

    k = cv2.waitKey(1) & 0xFF

    if k == 27 or k == ord('q'):
        break
            
# Releasing the video source.
cap.release()
# Destroying all the windows utilised by the app.
cv2.destroyAllWindows()

[1.2473661, 1.2569873, 1.2357959, 1.2830633, 1.2072278]
[1.2436334, 1.2546296, 1.2296028, 1.2790265, 1.2038109]
[1.2438802, 1.2550923, 1.2292236, 1.2798663, 1.2042859]
[1.2449254, 1.2537478, 1.2317072, 1.2804013, 1.2026502]
[1.2458845, 1.2550733, 1.2336241, 1.2819303, 1.2048359]
[1.2457905, 1.255173, 1.2329592, 1.2818114, 1.2040175]
[1.2453935, 1.2545784, 1.2330331, 1.2817262, 1.2040681]
[1.2454023, 1.2548946, 1.2327586, 1.2812555, 1.2045068]
[1.2441618, 1.2548863, 1.2313422, 1.2804757, 1.2048326]
[1.2419926, 1.2531691, 1.2294226, 1.2788495, 1.2035195]
[1.2421216, 1.2543745, 1.2285373, 1.279998, 1.2038865]
[1.247474, 1.2571032, 1.2359573, 1.2847835, 1.2068417]
[1.2474409, 1.2567801, 1.2369102, 1.2843302, 1.207229]
[1.2431358, 1.2538434, 1.2299932, 1.2804582, 1.2024117]
[1.2434009, 1.2540834, 1.2306973, 1.2809392, 1.2018653]
[1.2438743, 1.2529274, 1.2325226, 1.2803047, 1.2010939]
[1.2485433, 1.2573733, 1.238322, 1.2845376, 1.20608]
[1.2462159, 1.2566553, 1.2354281, 1.2830412, 1.2062192]